In [40]:
import pandas
import json
import requests
from requests.auth import HTTPDigestAuth
from requests_toolbelt.multipart import decoder
from urllib.parse import quote
from pandas.io.json import json_normalize

In [122]:
URI = 'http://localhost:8059/v1/eval'
USER = 'zomato-writer'
PASSWORD= 'zomato-writer'

In [123]:
def eval_query(query):
    url = URI
    data = {'xquery': query}
    headers = {'Accept': 'application/json'}
    auth=HTTPDigestAuth(USER,PASSWORD)
    result = requests.post(url, data=data, headers=headers,auth=auth,params=params)
    data = []
    df = None   
    status = str(result.status_code)
    if status == '200':
        if result.headers.get('Content-Type','').startswith("multipart/mixed"):
            data = get_multi_result(result)
        else:
            data.append(result.json())               
    else:
        print(status)
        error = result.json()["errorResponse"]["message"]
        print(error)
    df = pandas.DataFrame.from_dict(data, orient='columns')    
    return df

In [124]:
def get_multi_result(result):
    out = []
    multipart_data = decoder.MultipartDecoder.from_response(result)
    print("Returned {} results".format(len(multipart_data.parts)))
    for part in multipart_data.parts:
        ctype = part.headers[b'Content-Type'].decode('utf-8')
        raw = part.content
        data = json.loads(raw) if (ctype == 'application/json') else raw
        out.append(data) 
    return out 

In [125]:
query='''
import module namespace op="http://marklogic.com/optic" at "/MarkLogic/optic.xqy";

let $r := op:from-view("zomato", "reviews",'')
let $r-main := $r
  =>op:where(cts:element-geospatial-query(xs:QName('LatLon'),
  cts:circle(10,cts:point(53.474221,-2.241047))))
return
$r-main => op:result() 
'''
results = eval_query(query)

Returned 20 results


In [126]:
results.head()

,AverageCostForTwo,Country,Id,Latitude,Locality,Longitude,Name,RatingColor
0,50,United Kingdom,6801374,53.483500,Northern Quarter,-2.237333,Solita,Dark Green
1,55,United Kingdom,6800235,53.480833,Deansgate,-2.247333,The Grill On The Alley,Green
2,40,United Kingdom,6801963,53.484543,Northern Quarter,-2.237033,Almost Famous Burgers,Green
3,160,United Kingdom,6800782,53.477154,"The Midland, Deansgate",-2.245077,The French by Simon Rogan - The Midland,Green
4,35,United Kingdom,6800908,53.456000,Rusholme,-2.225333,Mughli,Dark Green


In [119]:
import folium
from folium.plugins import MarkerCluster

In [120]:
def plot_relatie_counts(rows):
    # generate a new map
    folium_map = folium.Map(location=[53.474221,-2.241047],
                            zoom_start=13,
                            width='100%',
                           height='100%')

    mc = MarkerCluster()
    # for each row in the data, add a cicle marker
    for index, row in rows.iterrows():
        lat =  row['Latitude']
        lon =  row['Longitude']
        rel = "<b>" + row['Name'] + "</b><br/>Average cost: £" + str(row['AverageCostForTwo']) + ""
        color= row['RatingColor']      
       
        mc.add_child(folium.Marker(location=[lat,  lon],popup=rel, icon=folium.Icon(color=color)))
 
    folium_map.add_child(mc)

                
    return folium_map

In [121]:
plot_relatie_counts(results)